In [5]:
%pip install faster-whisper pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp311-cp311-macosx_11_0_arm64.whl size=24131 sha256=15d453b3ee70b240347df33c1a0a1afb67484935a04cb550ff1088e3b875aaf6
  Stored in directory: /Users/gsampaio/Library/Caches/pip/wheels/80/b1/c1/67e4ef443de2665d86031d4760508094eab5de37d5d64d9c27
Successfully built pyaudio
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pyaudio
import wave
from faster_whisper import WhisperModel
import os
import time

/Users/gsampaio/redhat/ai/llm-agents/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
NEON_GREEN = "\033[92m"
RESET_COLOR = "\033[0m"


def transcribe_chunk(model, file_path):
    segments, info = model.transcribe(file_path, beam_size=5)
    transcription = " ".join(segment.text for segment in segments)
    return transcription


def record_chunk(p, stream, file_path, chunk_length=1):
    frames = []
    for _ in range(0, int(16000 / 1024 * chunk_length)):
        data = stream.read(1024)
        frames.append(data)

    wf = wave.open(file_path, "wb")
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(16000)
    wf.writeframes(b"".join(frames))
    wf.close()

In [12]:
# Choose your model settings
model_size = "small.en"
model = WhisperModel(model_size, device="cpu", compute_type="int8")

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=1024)

accumulated_transcription = ""  # Initialize an empty string to accumulate transcriptions

try:
    while True:
        chunk_file = "temp_chunk.wav"
        record_chunk(p, stream, chunk_file)
        transcription = transcribe_chunk(model, chunk_file)
        print(NEON_GREEN + transcription + RESET_COLOR)
        os.remove(chunk_file)
        
        # Append the new transcription to the accumulated transcription
        accumulated_transcription += transcription + " "

except KeyboardInterrupt:
    print("Stopping...")
    # Write the accumulated transcription to the log file
    with open("log.txt", "w") as log_file:
        log_file.write(accumulated_transcription)
finally:
    print("LOG:" + accumulated_transcription)
    stream.stop_stream()
    stream.close()
    p.terminate()

 Kala.  Kala.  Kala.  Kala.
LOG: Kala.  Kala.  Kala.  Kala. 


OSError: Stream not open